In [140]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import math
import pandas as pd
import pickle
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [104]:
obj_data_dir = "../../data/"
def save_obj(obj, name ):
    with open(obj_data_dir + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(obj_data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
# load data
# BasicFeatures = load_obj("pre_features")
training_edges=load_obj('SBdata')

In [68]:
data_dir = "../../node2vec/emb/"
cnt=0
flag=False
emb_dict = {}
with open(data_dir + "sb_data_edges.emd", "r") as f:
    read_rows = f.readlines()
    for row in read_rows:
        if cnt ==0:
            cnt+=1
            continue
        row_array = np.array(row.split())
        row_vector= [float(x) for x in row_array[1:]]
        emb_dict[int(row_array[0])]=row_vector

In [116]:
from sklearn.metrics.pairwise import cosine_similarity

# print(cosine_similarity([1, 0, -1], [-1,-1, 0]))
# array([[-0.5]])

training_edges_cosine = []
#  对training_data中的做cosine
mean =0
cnt=0
summation=0
for training_edge in training_edges:
    label = int(training_edge[1])
    source = int(training_edge[0][0])
    sink = int(training_edge[0][1]) 
    try:
        source_vector = np.array(emb_dict[source]).reshape(1, -1)
        sink_vector = np.array(emb_dict[sink]).reshape(1, -1)
        cosine_similarity_result = cosine_similarity(source_vector,sink_vector)[0]
        cnt +=1
        summation = cosine_similarity_result + summation
    except KeyError:
        cosine_similarity_result=None
    training_edges_cosine.append((source, sink, label, cosine_similarity_result))
mean = summation/cnt

In [120]:
mean[0]
# summation

array([0.00620332])

In [121]:
final_training_edges_cosine=[]
for edge in training_edges_cosine:
    if edge[3] is None:
        cosine_similarity_result=mean[0]
    else:
        cosine_similarity_result=edge[3][0]

    final_training_edges_cosine.append((edge[0], edge[1], edge[2], cosine_similarity_result))

In [136]:
final_training_data = pd.DataFrame(final_training_edges_cosine)
final_labels_df = final_training_data[2]
final_training_data_df = final_training_data.drop(columns=2)
# >>> df.drop(columns=['B', 'C'])


In [138]:
final_training_data_df=final_training_data_df

In [143]:
# 用logistic regression试试
X=final_training_data_df
# count=0
# get the data and label
y=final_labels_df

# training model
from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)
X_train, X_validation, y_train, y_validation  = train_test_split(X_t,y_t)
# Gridsearch settings
pipeline = Pipeline([
    ('clf', LogisticRegression())
])
parameters = {
       'clf__penalty': ('l1', 'l2'),
       'clf__C': (0.1, 1, 5),
 }
# 1. training_df_10w running
X_train = X_t
y_train = y_t
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,
   verbose=1, scoring='roc_auc', cv=3)
grid_search.fit(X_train, y_train)
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
predictions = grid_search.predict(X_test)
print('Accuracy:', accuracy_score(y_test, predictions))
print('Precision:', precision_score(y_test, predictions))
print('Recall:', recall_score(y_test, predictions))


Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   12.8s finished


Best score: 0.500
Best parameters set:
	clf__C: 0.1
	clf__penalty: 'l2'
Accuracy: 0.5130738420458046
Precision: 0.0
Recall: 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
filename='SBdata'
training_edges = load_obj(filename)

In [11]:
len(training_edges)

389478

In [12]:
ps_edges_for_node2vec=[]
for training_edge in training_edges:
    label = int(training_edge[1])
    source = int(training_edge[0][0])
    sink = int(training_edge[0][1])
    if label == 1:
        ps_edges_for_node2vec.append((source, sink))

In [13]:
len(ps_edges_for_node2vec)

189478

In [22]:
import csv
filename='ps_edges_for_node2vec_sb_data'
with open(filename + ".txt", 'w', encoding = 'utf8') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(ps_edges_for_node2vec)

In [ ]:
# END

In [25]:
import pandas as pd
def get_common_neighbours(node1, node2):
    try:
        n1 = pre_features[node1]
        n2 = pre_features[node2]
        common_neighors = list(set(n1[2]).intersection(n2[2]))
        return common_neighors
    except:
        return []

training_df = pd.DataFrame()

# prepare dataset for ANN
for edge in final_edges:  
#     print(source, sink, label)
    source = edge[0]
    sink = edge[1]
    label =edge[2]
    common_neighbours = get_common_neighbours(source, sink)
    if len(common_neighbours)>0:
        print('hello')
        print(len(common_neighbours))
    if len(common_neighbours)>=100:
        common_neighbours = common_neighbours[:100]
        
    else:
        pass
#     print(common_neighbours)
    common_neighbours.append(label)
    row_df = pd.DataFrame(common_neighbours)
    training_df = training_df.append(row_df)
    

In [19]:
filename='edges_10w'
with open(filename + ".txt", 'w', encoding = 'utf8') as f:
    f_csv = csv.writer(f)
    f_csv.writerows()

In [16]:
test=[1,2,3,4]
test[:2]
test.append

TypeError: can only concatenate list (not "int") to list

In [ ]:
# 算二级的common neighbour的个数
def 

In [15]:
import pandas as pd
# in-degree statistics
df = pd.DataFrame(BasicFeatures)
df =  df.T
df.rename(columns={0: 'num_of_neighbours',
                   1: 'log_num_of_neighbours',
                  2: 'list_of_neighbours',
                  3: 'list_of_in_neighbours',
                  4: 'num_of_in_neighbours',
                  5: 'list_of_out_neighbours',
                  6: 'num_of_out_neighbours'}, inplace=True)

In [45]:
df.loc[[2]]

,num_of_neighbours,log_num_of_neighbours,list_of_neighbours,list_of_in_neighbours,num_of_in_neighbours,list_of_out_neighbours,num_of_out_neighbours
2,2,0.910239,"[20388, 3360411]","[3360411, 20388]",2,[],0


In [67]:
new_df = df.loc[[2]]
new = new_df.append(df.loc[[1]])

In [69]:
new

,num_of_neighbours,log_num_of_neighbours,list_of_neighbours,list_of_in_neighbours,num_of_in_neighbours,list_of_out_neighbours,num_of_out_neighbours
2,2,0.910239,"[20388, 3360411]","[3360411, 20388]",2,[],0
1,3,0.721348,"[1247754, 2382107, 4588320]","[4588320, 1247754, 2382107]",3,[],0


In [72]:
new_df = pd.DataFrame()
# 找到所有的training data中有test data中有edge的边
for node_in_test_data in nodes_in_test_data:
    node_in_test_data = int(node_in_test_data)

    try:
        df_row = df.loc[[node_in_test_data]]
#         print(df_row)
        new_df = new_df.append(df_row)
#         print(new_df)
    except:
        continue

required_set = set()
for index, row in new_df.iterrows():
#     print(index)
#     print(row['list_of_neighbours'])
    required_set.add(index)
    for node in row['list_of_neighbours']:
        required_set.add(node)
len(required_set)
# 利用已经有的边生成新的训练集

data_dir = "../../data/"

with open(data_dir + "edges.txt", "r") as f:
    read_rows = f.readlines()


1222124

In [116]:
len(nodes_in_test_data)
nodes_in_test_data.astype(int8)

AttributeError: 'set' object has no attribute 'astype'

In [ ]:
rows = [i.split() for i in read_rows[0:]]
edges_of_all_test_nodes_related=list()

for row in rows:
    row[0] = int(row[0])
    row[1] = int(row[1])
    edge = (row[0], row[1])
    if str(row[0]) in nodes_in_test_data or str(row[1]) in nodes_in_test_data:
        edges_of_all_test_nodes_related.append(edge)

In [110]:
len(rows)

24004361

In [111]:
len(edges_of_all_test_nodes_related)

0

In [ ]:
# 生成negative sample
for 

In [5]:
import csv
import time
'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
#     headers = ['id','Prediction']
    with open(filename + str(nowtime()) + ".txt", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
#         f_csv.writerow(headers)
        for row in result:
            f_csv.writerow(result)

save_prediction_to_csv(Edges,'edges')

TypeError: writerows() takes no keyword arguments